## 매 시도마다 돌릴 것들

In [81]:
from google.colab import drive
drive.mount('/content/drive')
!pip install datasets --quiet

import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 데이터셋 경로 설정
data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from collections import Counter
from tqdm import tqdm
from collections import Counter
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import  AutoConfig
from scipy.special import softmax
import shutil

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 전처리

In [31]:
# 텍스트 데이터 로드
texts_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/texts.csv')
label_txt = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/label.txt', sep='\t', header=0)
label_txt.columns = ['ID', 'Annotator1', 'Annotator2', 'Annotator3']
label_jpg = label_txt.copy()

def pre_text(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

texts_df['Text'] = texts_df['Text'].apply(pre_text)

In [32]:
dic = {'positive': 1, 'neutral':0, 'negative': -1}
def pre_labeltext(text):
    new_text = text.split(",")[0]
    return dic[new_text]
label_txt['Annotator1'] = label_txt['Annotator1'].apply(pre_labeltext)
label_txt['Annotator2'] = label_txt['Annotator2'].apply(pre_labeltext)
label_txt['Annotator3'] = label_txt['Annotator3'].apply(pre_labeltext)
label_txt['label'] = label_txt['Annotator1'] + label_txt['Annotator2'] + label_txt['Annotator3']
label_txt['label'] = label_txt['label']/3
label_txt = label_txt.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [33]:
def pre_labeltext(text):
    new_text = text.split(",")[1]
    return dic[new_text]
label_jpg['Annotator1'] = label_jpg['Annotator1'].apply(pre_labeltext)
label_jpg['Annotator2'] = label_jpg['Annotator2'].apply(pre_labeltext)
label_jpg['Annotator3'] = label_jpg['Annotator3'].apply(pre_labeltext)
label_jpg['label'] = label_jpg['Annotator1'] + label_jpg['Annotator2'] + label_jpg['Annotator3']
label_jpg['label'] = label_jpg['label']/3
label_jpg = label_jpg.drop(['Annotator1', 'Annotator2', 'Annotator3'], axis=1)

In [35]:
# 라벨 데이터와 텍스트 데이터 병합
merged_text = pd.merge(label_txt, texts_df, on='ID')

## 데이터 라벨링-모델 일치율 확인(만족스럽다)

In [60]:
texts = merged_text['Text'].tolist()
actual_labels = merged_text['label'].tolist()
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [61]:
# 배치 사이즈 설정
batch_size = 16
sentiment_scores = []

for i in tqdm(range(0, len(texts), batch_size)):
    batch_texts = texts[i:i+batch_size]
    encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')
    encoded_input.to(device)

    with torch.no_grad():
        outputs = model(**encoded_input)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    scores = probabilities.cpu().numpy()

    # 감성 점수 계산: (긍정 확률 - 부정 확률)
    pos_probs = scores[:, 2]  # 긍정 확률
    neg_probs = scores[:, 0]  # 부정 확률
    sentiments = pos_probs - neg_probs  # 감성 점수
    sentiment_scores.extend(sentiments)

# 실제 라벨과 예측된 감성 점수 변환
actual_labels = np.array(actual_labels)
sentiment_scores = np.array(sentiment_scores)

# L2 Loss 계산
l2_loss = np.mean((sentiment_scores - actual_labels) ** 2)
print(f"L2 Loss: {l2_loss}")

# 상관계수 계산
correlation = np.corrcoef(actual_labels, sentiment_scores)[0, 1]
print(f"상관계수: {correlation}")

100%|██████████| 1225/1225 [00:18<00:00, 66.46it/s]

L2 Loss: 0.22832325209626356
상관계수: 0.5043935582499393


In [65]:
count = 0
for i in range (19600):
    if sentiment_scores[i] * actual_labels[i] < 0:
        count+=1
count

1704

In [64]:
sentiment_scores[0]

0.15349671

## 이미지 설명

In [ ]:
#자기 전에 켜 둘 것
source_dir = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data'

# 대상 데이터 경로 (Colab 로컬 스토리지)
target_dir = '/content/data'

# 대상 폴더가 없으면 생성
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

loader = label_txt['ID'].tolist()

for filename in tqdm(loader):
    filename_with_ext = f"{filename}.jpg"
    src_file = os.path.join(source_dir, filename_with_ext)
    dst_file = os.path.join(target_dir, filename_with_ext)

    try:
        shutil.copy2(src_file, dst_file)
    except Exception as e:
        print(f"파일 복사 중 오류 발생: {filename_with_ext}, 오류: {e}")

  1%|          | 139/19600 [00:46<1:42:06,  3.18it/s]

In [76]:
scores = np.array(sentiment_scores)
sim = label_jpg['label'] * label_txt['label'] > 0
label_jpg['pred'] = scores
input_df = label_jpg[sim]

In [88]:
loader[0]

2499